# Smoothing

這邊我們先給各位看一下使用numpy手刻完成一些最簡單的forcasting

numpy手刻只是為了讓同學了解這些算法在使用python實現的方式，後面我們會附上一些簡單使用的套件，以後在使用時直接call套件的class或者funciton就可以簡單實現演算法

課程包含以下內容:
- Moving Average
- Simple Exponential Smoothing
- Finite Difference

#### **開始前請先安裝或import基本套件**
#### **若使用Jupyter Notebook開啟請轉成tree view方便顯示plotly出來的圖**

In [ ]:
! pip install --user plotly

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt

from plotly import express as px

**另外我們也先準備一個畫圖的function，我們不會放重點在這邊但後面會用它來看一些time series處理的過程**

In [ ]:
def plot_series(time, series, start=0, end=None, labels=None, title=None):
    #  Visualizes time series data
    # Args:
    #  time (array of int) - 時間點, 長度為T
    #  series (list of array of int) - 時間點對應的資料列表，列表內時間序列數量為D，
    #                                  每筆資料長度為T，若非為列表則轉為列表
    #  start (int) - 開始的資料序(第幾筆)
    #  end (int) -   結束繪製的資料序(第幾筆)
    #  labels (list of strings)- 對於多時間序列或多維度的標註
    #  title (string)- 圖片標題

    # 若資料只有一筆，則轉為list
    if type(series) != list:
        series = [series]

    if not end:
        end = len(series[0])

    if labels:
        # 設立dictionary, 讓plotly畫訊號線時可以標註label
        dictionary = {"time": time}
        for idx, l in enumerate(labels):
            # 截斷資料，保留想看的部分，並分段紀錄於dictionary中
            dictionary.update({l: series[idx][start:end]})
        # 畫訊號線
        fig = px.line(dictionary,
                      x="time",
                      y=list(dictionary.keys())[1:],
                      width=1000,
                      height=400,
                      title=title)
    else:
        # 畫訊號線
        fig = px.line(x=time, y=series, width=1000, height=400, title=title)
    fig.show()

這邊也附上我們的toy data產生器

In [ ]:
def trend(time, slope=0):
    # 產生合成水平直線資料，其長度與時間等長，直線趨勢與設定slope相同
    # Args:
    #  time (array of int) - 時間點, 長度為T
    #  slope (float) - 設定資料的傾斜程度與正負
    # Returns:
    #  series (array of float) -  產出slope 與設定相同的一條線

    series = slope * time

    return series


def seasonal_pattern(season_time, pattern_type='triangle'):
    # 產生某個特定pattern，
    # Args:
    #  season_time (array of float) - 周期內的時間點, 長度為T
    #  pattern_type (str) -  這邊提供triangle與cosine
    # Returns:
    #  data_pattern (array of float) -  根據自訂函式產出特定的pattern

    # 用特定function生成pattern
    if pattern_type == 'triangle':
        data_pattern = np.where(season_time < 0.5,
                                season_time*2,
                                2-season_time*2)
    if pattern_type == 'cosine':
        data_pattern = np.cos(season_time*np.pi*2)

    return data_pattern


def seasonality(time, period, amplitude=1, phase=30, pattern_type='triangle'):
    # Repeats the same pattern at each period
    # Args:
    #   time (array of int) - 時間點, 長度為T
    #   period (int) - 週期長度，必小於T
    #   amplitude (float) - 序列幅度大小
    #   phase (int) - 相位，為遞移量，正的向左(提前)、負的向右(延後)
    #   pattern_type (str) -  這邊提供triangle與cosine
    # Returns:
    #   data_pattern (array of float) - 有指定周期、振幅、相位、pattern後的time series

    # 將時間依週期重置為0
    season_time = ((time + phase) % period) / period

    # 產生週期性訊號並乘上幅度
    data_pattern = amplitude * seasonal_pattern(season_time, pattern_type)

    return data_pattern


def noise(time, noise_level=1, seed=None):
    # 合成雜訊，這邊用高斯雜訊，機率密度為常態分布
    # Args:
    #   time (array of int) - 時間點, 長度為T
    #   noise_level (float) - 雜訊大小
    #   seed (int) - 同樣的seed可以重現同樣的雜訊
    # Returns:
    #   noise (array of float) - 雜訊時間序列

    # 做一個基於某個seed的雜訊生成器
    rnd = np.random.RandomState(seed)

    # 生與time同長度的雜訊，並且乘上雜訊大小 (不乘的話，標準差是1)
    noise = rnd.randn(len(time)) * noise_level

    return noise


def toy_generation(time=np.arange(4 * 365),
                   bias=500.,
                   slope=0.1,
                   period=180,
                   amplitude=40.,
                   phase=30,
                   pattern_type='triangle',
                   noise_level=5.,
                   seed=2022):
    signal_series = bias\
                  + trend(time, slope)\
                  + seasonality(time,
                                period,
                                amplitude,
                                phase,
                                pattern_type)
    noise_series = noise(time, noise_level, seed)

    series = signal_series+noise_series
    return series

我們附上最naive的k-step ahead prediction

In [ ]:
def k_step_ahead(data, k):
    # 產生k-step ahead預測
    # Args:
    #  data (array of float) - 輸入資料
    # Returns:
    #  forcast (array of float) -  k-step ahead預測結果

    forcast = data[:-k]
    return forcast

也附上評估的function

In [ ]:
def MAE(pred, gt):
    # 計算Mean Absolute Error
    # Args:
    #  pred (array of float) - 預測資料
    #  gt (array of float) - 答案資料
    # Returns:
    #  計算結果 (float)
    return abs(pred-gt).mean()


def MSE(pred, gt):
    # 計算Mean Square Error
    # Args:
    #  pred (array of float) - 預測資料
    #  gt (array of float) - 答案資料
    # Returns:
    #  計算結果 (float)
    return pow(pred-gt, 2).mean()


def R2(pred, gt):
    # 計算R square score
    # Args:
    #  pred (array of float) - 預測資料
    #  gt (array of float) - 答案資料
    # Returns:
    #  計算結果 (float)
    return 1-pow(pred-gt, 2).sum()/pow(gt-gt.mean(), 2).sum()

另外我們記得生成資料

In [ ]:
# 先合成資料
time = np.arange(4 * 365)  # 定義時間點
series_sample = toy_generation(time)  # 這就是我們合成出來的資料


# 最簡單直接取前後，並且時間點也記得要切，我們直接立個function
def split(x, train_size):
    return x[..., :train_size], x[..., train_size:]


time_train, time_test = split(time, 365*3)
series_train, series_test = split(series_sample, 365*3)

## Moving Average

Moving average 把前K個時間點的資料做平均，使用這平均來預測下筆資料。
<img src="https://i.imgur.com/iPt1Iwq.png" width=400 align="right">

實現方式有很多種，包含對每個時間點慢慢使用平均、或者使用捲積運算等等。

因為是對未來的預測，所以會希望前K天[t-K,...,t-2,t-1]資料預測該時間點(t)資料

In [ ]:
# 用捲積做看看5天平均的moving average
K = 5
kernel = np.ones(K)/K  # 對K個1做捲積在除以K就是做移動的平均
# 最開始會取最前K筆做第一次平均，最後一次算會包含最後一筆，所以要去掉
forcast = np.convolve(series_train, kernel, mode='valid')[:-1]
ground_truth_for_view = series_train[K:]  # 去掉前K日資料
time_for_view = time_train[K:]

plot_series(time_for_view,
            [forcast, ground_truth_for_view],
            labels=['prediction', 'ground truth'])

可以把它包起來

In [ ]:
def moving_average(data, K):
    # 產生前k個資料的moving average預測
    # Args:
    #  data (array of float) - 輸入資料
    #  K (float) - 每次要平均的資料筆數
    # Returns:
    #  forcast (array of float) -  moving average預測結果

    forcast = np.convolve(data, np.ones(K)/K, mode='valid')[:-1]
    return forcast

## Simple Exponential Smoothing

若考慮到更遠古的資料對現在的預測來講，重要性應該要下降，可以使用Simple Exponential Smoothing。

$$F_{t}=\alpha y_{t-1}+(1-\alpha) F_{t-1}$$
<img src=https://i.imgur.com/Wfn79zs.png width=600>

它會將前一時間點[t-1]的預測與前個時間點的資料做加權和，而前一個時間點的預測亦是從前前時間點[t-2]的資料來的，當權重$1-\alpha$介於$[0,1)$時將會使遠古的資料影響力隨著時間逐漸趨於0。

$$F_{t}=\alpha y_{t-1}+\alpha(1-\alpha) y_{t-2}+\alpha(1-\alpha)^2 y_{t-3}+...+\alpha(1-\alpha)^{t-1} y_{0}$$



In [ ]:
# 我們試著用for迴圈推每次的結果
alpha = 0.9
forcast = []
for t in range(1, len(series_train)):
    if forcast:
        forcast.append(alpha*series_train[t-1]+alpha*(1-alpha)*forcast[-1])
    else:
        forcast = [alpha*series_train[t-1]]
forcast = np.array(forcast)

ground_truth_for_view = series_train[1:]  # 去掉前1日資料
time_for_view = time_train[1:]

plot_series(time_for_view,
            [ground_truth_for_view, forcast],
            labels=['ground truth', 'ses'])

試著show 一下weight項，我們看
$$[\alpha,\alpha(1-\alpha),\alpha(1-\alpha)^2,\alpha(1-\alpha)^3,...]$$
$$=\alpha*[(1-\alpha)^0,(1-\alpha)^1,(1-\alpha)^2,...]$$

In [ ]:
alpha = 0.9
# get [0,1,2,3,...]
exp = np.arange(0, len(series_train))

# get [(1-alpha),(1-alpha),(1-alpha),...]
a = np.repeat(1-alpha, len(series_train))

# get power of (1-alpha)
weight = np.power(a, exp)*alpha

In [ ]:
# 可調整alpha嘗試看看，大概前幾個weight就太小了，後面基本可忽略
weight[:10]

In [ ]:
plt.figure(figsize=(10, 2))
plt.bar(range(1, 11), weight[:10], tick_label=[str(n) for n in range(1, 11)])

In [ ]:
def ses(data, k, alpha=0.95):
    # 產生關於K個時間點後的Simple Exponential Smoothing預測
    # Args:
    #  data (array of float) - 輸入資料
    #  alpha (float) - 衰減係數，決定過往資料比重
    #  k (float) - 要往後預測的資料筆數
    # Returns:
    #  forcast (array of float) -  Simple Exponential Smoothing預測結果

    forcast = []
    for t in range(1, len(data)):
        if forcast:
            forcast.append(alpha*data[t-1]+alpha*(1-alpha)*forcast[-1])
        else:
            forcast = [data[t-1]]
    return np.array(forcast)[:-K+1]

## Comparison

把前述幾個演算法互相比較一下

In [ ]:
# 我們把它跟k_step比較一下結果
K = 5
forcast_d = {
    'ground_truth': series_train[K:],  # 去掉前K日資料
    'k_step': k_step_ahead(series_train, K),
    'ses': ses(series_train, K),
    'ma': moving_average(series_train, K)
}

time_for_view = time_train[K:]

plot_series(time_for_view,
            list(forcast_d.values()),
            labels=list(forcast_d.keys()))

In [ ]:
for method, pred in [*forcast_d.items()][1:]:
    print(f'''{method}:
    -MAE:{MAE(pred[5:], forcast_d['ground_truth'][5:])}
    -MSE:{MSE(pred[5:], forcast_d['ground_truth'][5:])}
    -R2:{R2(pred[5:], forcast_d['ground_truth'][5:])}''')

目前的資料中因為沒有除卻trend與bias的關係，k-step與ses的效果沒辦法發揮，後面做到linear regression後可再試試。

### 模擬de-trend後的結果

In [ ]:
mytrend = trend(time_train, 0.1)+500.
series_train2 = series_train-mytrend

K = 180

forcast_d = {
    'ground_truth': series_train2[K:],  # 去掉前K日資料
    'k_step': k_step_ahead(series_train2, K),
    'ses': ses(series_train2, K, alpha=0.7),
    'ma': moving_average(series_train2, K)
}

time_for_view = time_train[K:]

plot_series(time_for_view,
            list(forcast_d.values()),
            labels=list(forcast_d.keys()))

In [ ]:
for method, pred in [*forcast_d.items()][1:]:
    print(f'''{method}:
    -MAE:{MAE(pred[5:], forcast_d['ground_truth'][5:])}
    -MSE:{MSE(pred[5:], forcast_d['ground_truth'][5:])}
    -R2:{R2(pred[5:], forcast_d['ground_truth'][5:])}''')

可以看出來，在去掉trend與bias後，使用k-step與ses推估已知周期(180天)後的序列，效果會比做180天的moving average好很多。

## Finite Difference

當我們預期時間序列在短時間內有比較大的變動時，我們常常不會直接predict序列本身而是序列的有限差分(finite difference)。
$$d_0=not exist$$
$$d_t=y_t-y_{t-1}$$

這樣會使得序列中最高頻率的特性被抓出來，低頻的trend與bias也可以透過如此方法被去掉，再預測時會比較方便。

不過有些頻率太低的訊號特性也會同時被削減。

而且這樣的訊號不是元訊號，預測完還須還原就是了。

In [ ]:
def dif(x):
    return x[1:]-x[:-1]
plot_series(time_train, series_train, title='Original')
diff = dif(series_train)
plot_series(time_train[1:], diff, title='Difference')

### Recover Finite Difference
既然是差分，只要給予原本訊號中最前面的資料，再一一做summary就可以復原。
$$r_0=y_0$$
$$r_t=y_{t-1}+d_{t}$$

In [ ]:
plot_series(time_train,
            [series_train, np.cumsum([series_train[0], *diff])],
            labels=['ground truth', 'recovered'])

In [ ]:
diff = dif(series_train)
K = 5

forcast_d = {
    'ground_truth': diff[K:],  # 去掉前K日資料
    'k_step': k_step_ahead(diff, K),
    'ses': ses(diff, K, alpha=0.7),
    'ma': moving_average(diff, K)
}

time_for_view = time_train[K+1:]

plot_series(time_for_view,
            list(forcast_d.values()),
            labels=list(forcast_d.keys()))

例如目前我們的序列是180天的周期，形成差分序列時已經把大部分的內容削去幾乎只剩noise

所以這種大時間尺度序列的其實用difference可能model不出什麼東西

In [ ]:
for method, pred in [*forcast_d.items()][1:]:
    print(f'''{method}:
    -MAE:{MAE(pred[5:], forcast_d['ground_truth'][5:])}
    -MSE:{MSE(pred[5:], forcast_d['ground_truth'][5:])}
    -R2:{R2(pred[5:], forcast_d['ground_truth'][5:])}''')

## Call Function from Module

### Moving average

Moving Average因為很簡單，所以沒有在大的module裡面有工具。

但在Pandas有```Series.rolling(window).mean()```可以用，其中window就是window size，一次要平均的資料比數

(不過先存成Series會比較耗資源，可能自己用前面方法寫結果還跑得比較快)

In [ ]:
import pandas as pd
forcast = pd.Series(series_train).rolling(5).mean()

plot_series(time_train,
            [series_train, forcast],
            labels=['ground truth', f'ma,window=5'])

### SES

SES較複雜一點，在statsmodels有工具，並且具有training alpha的功能以及做完detrend

In [ ]:
from statsmodels.tsa.api import SimpleExpSmoothing as SES
# smoothing_level: alpha
# model = SES(series_train).fit(smoothing_level=0.2, optimized=False)
model = SES(series_train).fit()
#  後面fit那邊不填入任何內容可以自動train alpha

forcast = model.fittedvalues

plot_series(time_train,
            [series_train, forcast],
            labels=['ground truth',
                    f'ses, alpha={model.params["smoothing_level"]}'])

# 其中 detrend加recover它都幫我們做好了，所以不受trend影響

In [ ]:
# 預測在測試集上
model = SES(series_train).fit()

forcast = SES(series_test).fit(
    smoothing_level=model.params["smoothing_level"],
    optimized=False).fittedvalues

plot_series(time_test,
            [series_test, forcast],
            labels=['ground truth',
                    f'ses,alpha={model.params["smoothing_level"]}'])

### Diff
Finite Difference也很簡單，不過可以使用```np.diff(series,n)```來做。

series就是目標序列，n代表做幾次差分，可以做超過一階的差分

In [ ]:
print(np.diff([1, 2, 3, 4, 5], n=1))
print(np.diff([1, 2, 3, 4, 5], n=2))